Evaluation of models on separate datasets split by years.

In [ ]:
import os
import gdown
import shutil
import numpy as np
import pandas as pd
import pathlib
from glob import glob
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from tensorflow.keras.utils import save_img, img_to_array, array_to_img, load_img, image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import register_keras_serializable
# from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve, roc_curve, auc

In [ ]:
from data_processing_utils import show_dirs_len, prepocess_to_crop, separate_vignette_images, crop_and_save

from models_processing_utils import add_files, sample_and_copy_files,\
                                    plot_scores, plot_loss, plot_confusion_matrix, fbeta_metric,\
                                    get_predictions_df, get_predictions_df2, show_predicted,\
                                    scan_dataset, preprocess_image, split_by_class
from models_processing_utils import ModelEvaluating, ModelPredicting

## Формування робочого датасету

Завантаження джерела

In [ ]:
# https://drive.google.com/file/d/1Wn-z5ozhs94Pfgrix70CiCcQGZdTXh3z/view?usp=sharing
file_id = "1Wn-z5ozhs94Pfgrix70CiCcQGZdTXh3z"
url = f"https://drive.google.com/uc?id={file_id}"
output = "2020_supplemented_cropped.zip"

gdown.download(url, output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1Wn-z5ozhs94Pfgrix70CiCcQGZdTXh3z
From (redirected): https://drive.google.com/uc?id=1Wn-z5ozhs94Pfgrix70CiCcQGZdTXh3z&confirm=t&uuid=4644088f-736b-4957-82ce-a405c6eaa734
To: /content/2020_supplemented_cropped.zip
100%|██████████| 1.59G/1.59G [00:35<00:00, 44.1MB/s]


'2020_supplemented_cropped.zip'

In [ ]:
!unzip -q 2020_supplemented_cropped.zip

In [ ]:
os.makedirs('2020_supplemented_cropped', exist_ok=True)
!mv /content/content/2020_supplemented/* /content/2020_supplemented_cropped/

In [ ]:
show_dirs_len(pathlib.Path("/content/2020_supplemented_cropped"))

.: 2
test: 2
test/other: 10638
test/melanoma: 1587
train: 2
train/other: 32112
train/melanoma: 5093


In [ ]:
# shutil.rmtree("/content/2020_supplemented_cropped")
# shutil.rmtree("/content/2019_all_croppedtr")

In [ ]:
# data_dir = pathlib.Path("/content/2020_supplemented_cropped")

Завантажити зображення з набору 2019 року з видаленими чорними краями в тренувальному наборі.

In [ ]:
# https://drive.google.com/file/d/1Qo98PZ2fsgsp-nEG6DW4XX5NtUb16xoi/view?usp=sharing
file_id = "1Qo98PZ2fsgsp-nEG6DW4XX5NtUb16xoi"
url = f"https://drive.google.com/uc?id={file_id}"
output = "2019_all_croppedtr.zip"

gdown.download(url, output, quiet=False)
!unzip -q 2019_all_croppedtr.zip

Downloading...
From (original): https://drive.google.com/uc?id=1Qo98PZ2fsgsp-nEG6DW4XX5NtUb16xoi
From (redirected): https://drive.google.com/uc?id=1Qo98PZ2fsgsp-nEG6DW4XX5NtUb16xoi&confirm=t&uuid=4078266e-3210-4a45-9006-91848d2cd5b0
To: /content/2019_all_croppedtr.zip
100%|██████████| 1.26G/1.26G [00:23<00:00, 53.7MB/s]


In [ ]:
show_dirs_len(pathlib.Path("/content/2019_all_croppedtr"))

.: 2
test: 2
test/other: 6911
test/melanoma: 1327
train: 2
train/other: 20768
train/melanoma: 4513


Завантажити набір 2 (2019+2020) для отримання валідаційних зображень

In [ ]:
# весь cropped датасет без аугментації
# https://drive.google.com/file/d/1vIdcm_P_m5jJxWtidqct1iy4VfiJfO4Y/view?usp=sharing
file_id = "1vIdcm_P_m5jJxWtidqct1iy4VfiJfO4Y"
url = f"https://drive.google.com/uc?id={file_id}"
output = "full_ds_croppedtr_no_augm.zip"

gdown.download(url, output, quiet=False)

!unzip -q full_ds_croppedtr_no_augm.zip

Downloading...
From (original): https://drive.google.com/uc?id=1vIdcm_P_m5jJxWtidqct1iy4VfiJfO4Y
From (redirected): https://drive.google.com/uc?id=1vIdcm_P_m5jJxWtidqct1iy4VfiJfO4Y&confirm=t&uuid=95b2e8ee-7e80-4baf-8f03-9b434b0612ee
To: /content/full_ds_croppedtr_no_augm.zip
100%|██████████| 2.08G/2.08G [00:24<00:00, 84.2MB/s]


Розділити набори, виділити дані, що були на валідації

З набору 2019 вилучити окремо валідаційні дані з train, з набору 2020 train/melanoma та test/melanoma вилучити 2019 та валідаційні дані з train

In [ ]:
folder_validation = "/content/full_ds_croppedtr_no_augm/validation"
class_names = {'other': 0, 'melanoma': 1}
pathes, labels = scan_dataset(folder_validation, class_names)
files = [os.path.basename(path) for path in pathes]
valid_df = pd.DataFrame({
                      'file_name': files,
                      'true_label': labels})
valid_df.to_csv("validation_data_set2.csv")

Found 0 files of class other
Found 0 files of class melanoma


In [ ]:
# valid_df = pd.read_csv('/content/validation_data_set2.csv')

In [ ]:
def extract_validation_data(folder, valid_df, class_names):
  for k in class_names.keys():
    source_dir = os.path.join(folder,"train", k)
    dest_dir = os.path.join(folder,"validation", k)
    os.makedirs(dest_dir, exist_ok=True)
    for filename in os.listdir(source_dir):
      if filename in valid_df['file_name'].values:
        shutil.move(os.path.join(source_dir, filename), os.path.join(dest_dir,filename))
    print(f"validation {k}: {len(os.listdir(dest_dir))}")


In [ ]:
def remove_intersection(folder_clean, folder_compare, class_names, subfolder=''):
  for k in class_names.keys():
      dir1 = os.path.join(folder_clean, subfolder, k)
      dir2 = os.path.join(folder_compare,subfolder, k)
      files1 = set(os.listdir(dir1))
      files2 = set(os.listdir(dir2))
      common_files = files1.intersection(files2)
      for file in common_files:
        src_path = os.path.join(dir1, file)
        if os.path.exists(src_path):
          os.remove(src_path)

In [ ]:
folder_20 = "/content/2020_supplemented_cropped"
folder_19 = "/content/2019_all_croppedtr"

# очистити датасет 2020 від даних інших років
remove_intersection(folder_20, folder_19, class_names, subfolder='train')
remove_intersection(folder_20, folder_19, class_names, subfolder='test')
print("2020:")
show_dirs_len(folder_20)
print("2019:")
show_dirs_len(folder_19)
# виділити валідаційні дані, що використовувались раніше
extract_validation_data(folder_20, valid_df, class_names)
extract_validation_data(folder_19, valid_df, class_names)

2020:
.: 2
test: 2
test/other: 10638
test/melanoma: 260
train: 2
train/other: 32112
train/melanoma: 586
2019:
.: 2
test: 2
test/other: 6911
test/melanoma: 1327
train: 2
train/other: 20768
train/melanoma: 4513
validation other: 0
validation melanoma: 0
validation other: 0
validation melanoma: 0


In [ ]:
valid_df

,file_name,true_label


In [ ]:
'ISIC_0025745.jpg' in valid_df['file_name'].values

True

## Оцінка моделей на окремих датасетах

In [ ]:
@register_keras_serializable()
class ConvDepthwiseConvBlock(layers.Layer):
    def __init__(self, filters, scale=1, kernel_size=3, strides=1, activation="silu", kernel_regularizer=None, name="ConvDeptConv", **kwargs):
        super().__init__(name=name, **kwargs)
        self.filters = filters
        self.scale = scale
        self.kernel_size = kernel_size
        self.strides = strides
        self.activation = keras.activations.get(activation)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)

        self.conv1 = None
        self.bn1 = layers.BatchNormalization()
        self.act1 = layers.Activation(self.activation)

        self.dw_conv = None
        self.bn2 = layers.BatchNormalization()
        self.act2 = layers.Activation(self.activation)

        self.conv2 = None
        self.bn3 = layers.BatchNormalization()
        self.act3 = layers.Activation(self.activation)

    def build(self, input_shape):
        self.conv1 = layers.Conv2D(filters=self.filters*self.scale,
                                   kernel_size=self.kernel_size,
                                   strides=self.strides,
                                   padding="same",
                                   kernel_regularizer=self.kernel_regularizer)

        self.dw_conv = layers.DepthwiseConv2D(kernel_size=self.kernel_size,
                                              strides=1,
                                              padding="same",
                                              depthwise_regularizer=self.kernel_regularizer)

        self.conv2 = layers.Conv2D(filters=self.filters,
                                   kernel_size=self.kernel_size,
                                   strides=1, padding="same",
                                   kernel_regularizer=self.kernel_regularizer)
        super().build(input_shape)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.act1(x)

        x = self.dw_conv(x)
        x = self.bn2(x)
        x = self.act2(x)

        x = self.conv2(x)
        x = self.bn3(x)
        x = self.act3(x)
        return x

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "scale": self.scale,
            "kernel_size": self.kernel_size,
            "strides": self.strides,
            "kernel_regularizer": keras.regularizers.serialize(self.kernel_regularizer),
        })
        return config

In [ ]:
@register_keras_serializable()
class CBAM(layers.Layer):
    def __init__(self, reduction_ratio=8,  kernel_size=7, name="cbam", kernel_regularizer=None, **kwargs):
        super().__init__(name=name, **kwargs)
        self.reduction_ratio = reduction_ratio
        self.kernel_size = kernel_size
        self.kernel_regularizer=keras.regularizers.get(kernel_regularizer)

    def build(self, input_shape):
        channel = input_shape[-1]

        # Channel Attention
        self.global_avg_pool = layers.GlobalAveragePooling2D() #squeeze in one vector (batch_size, channels), avg each feature map (channel) of all pixls
        self.global_max_pool = layers.GlobalMaxPooling2D()

        #multy-layers perceptron
        self.shared_mlp = tf.keras.Sequential([
            layers.Dense(channel // self.reduction_ratio, activation='relu', kernel_regularizer=self.kernel_regularizer),
            layers.Dense(channel, kernel_regularizer=self.kernel_regularizer)
        ])

        self.reshape = layers.Reshape((1, 1, channel))

        # Spatial Attention
        self.conv_spatial = layers.Conv2D(filters=1,
                                  kernel_size=self.kernel_size,
                                  strides=1,
                                  padding='same',
                                  activation='sigmoid',
                                  kernel_regularizer=self.kernel_regularizer)

    def call(self, inputs, return_attention=False):
        # Channel Attention
        avg_pool = self.global_avg_pool(inputs)
        max_pool = self.global_max_pool(inputs)

        avg_full_conn = self.shared_mlp(avg_pool)
        max_full_conn = self.shared_mlp(max_pool)

        channel_attention = tf.keras.activations.sigmoid(avg_full_conn + max_full_conn)
        channel_attention = self.reshape(channel_attention)
        x = inputs * channel_attention  # Broadcasted multiplication

        # Spatial Attention
        avg_pool_spatial = tf.reduce_mean(x, axis=-1, keepdims=True) # карта ознак (batch_size, height, width, 1), avg of all channels
        max_pool_spatial = tf.reduce_max(x, axis=-1, keepdims=True)
        spatial_attention = tf.concat([avg_pool_spatial, max_pool_spatial], axis=-1)

        spatial_attention = self.conv_spatial(spatial_attention)
        x = x * spatial_attention  # Element-wise multiplication

        if return_attention:
            return x, channel_attention, spatial_attention
        return x

    def get_config(self):
        config = super().get_config()
        config.update({
            "reduction_ratio": self.reduction_ratio,
            "kernel_regularizer": keras.regularizers.serialize(self.kernel_regularizer),
            "kernel_size": self.kernel_size,
        })
        return config


In [ ]:
from keras.applications.densenet import preprocess_input as densenet_preprocess
from keras.applications.efficientnet_v2 import preprocess_input as efficientnet_v2_preprocess
from keras.applications.xception import preprocess_input as xception_preprocess

In [ ]:
model_path1 = '/content/ft-EfficientNetV2B0_04-23-10.keras'
model_path2 = '/content/ft-EfficientNetV2B0_04-29-19.keras'
model_path3 = '/content/ft-EfficientNetV2B0_04-12-10_last.keras'
model_path_custom = '/content/Custom_CNN_05-11-17_.keras'
model_path_dense = '/content/ft-DenseNet121_05-05-17.keras'
model_path_xception = '/content/ft-Xception_05-20-16.keras'

# efficientnet_v2_preprocess, densenet_preprocess, xception_preprocess
model_1 = keras.models.load_model(model_path1, custom_objects={"preprocess_input": efficientnet_v2_preprocess})
model_2 = keras.models.load_model(model_path2, custom_objects={"preprocess_input": efficientnet_v2_preprocess})
model_3 = keras.models.load_model(model_path3, custom_objects={"preprocess_input": efficientnet_v2_preprocess})
model_custom = keras.models.load_model(model_path_custom, custom_objects={"ConvDeptConv": ConvDepthwiseConvBlock, "cbam": CBAM})
model_dense = keras.models.load_model(model_path_dense, custom_objects={"preprocess_input": densenet_preprocess})
model_xception = keras.models.load_model(model_path_xception, custom_objects={"preprocess_input": xception_preprocess})

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 202 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
models = [model_1, model_2, model_3, model_custom, model_dense, model_xception]

In [ ]:
IM_SIZE = 256
BATCH_SIZE = 32

In [ ]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
]

In [ ]:
test_ds19 = tf.keras.utils.image_dataset_from_directory(
  pathlib.Path(folder_19) / "test",
  class_names=["other", "melanoma"],
  seed=123,
  # image_size=(IM_SIZE, IM_SIZE),
  batch_size=BATCH_SIZE)

# test_ds19 = test_ds19.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 8238 files belonging to 2 classes.


In [ ]:
test_ds20 = tf.keras.utils.image_dataset_from_directory(
  pathlib.Path(folder_20) / "test",
  class_names=["other", "melanoma"],
  seed=123,
  # image_size=(IM_SIZE, IM_SIZE),
  batch_size=BATCH_SIZE)

# test_ds20 = test_ds20.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 10898 files belonging to 2 classes.


In [ ]:
MODEL_NAMES = [f"EfficientNetV2B0_04-23-10",
              f"EfficientNetV2B0_04-29-19",
              f"EfficientNetV2B0_04-12-10_last",
              f"Custom_CNN_05-11-17_",
              f"DenseNet121_05-05-17",
              f"Xception_05-20-16"]

In [ ]:
# ! Перезапускати
model_index = 3
MODEL_NAME = MODEL_NAMES[model_index]

### Dataset 2020 evaluation

In [ ]:
for model, model_name in zip(models, MODEL_NAMES):
  print(model_name)
  results = ModelEvaluating(test_ds20,
                           model, model_name=model_name,
                           metrics=METRICS)
  test_ds20.unbatch().batch(BATCH_SIZE)

EfficientNetV2B0_04-23-10
341/341 ━━━━━━━━━━━━━━━━━━━━ 33s 64ms/step - accuracy: 0.9773 - auc: 0.8663 - fn: 103.9795 - fp: 24.2632 - loss: 0.0875 - prc: 0.3029 - precision: 0.5550 - recall: 0.2121 - tn: 5331.5029 - tp: 28.0789
loss :  0.08601237088441849
accuracy :  0.9764177203178406
precision :  0.5128205418586731
recall :  0.23076923191547394
tp :  60.0
fp :  57.0
tn :  10581.0
fn :  200.0
auc :  0.885644257068634
prc :  0.29702475666999817
f1_beta :  0.2592912731696192


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

EfficientNetV2B0_04-29-19
341/341 ━━━━━━━━━━━━━━━━━━━━ 25s 52ms/step - accuracy: 0.9625 - auc: 0.8822 - fn: 78.0877 - fp: 125.9444 - loss: 0.1089 - prc: 0.3564 - precision: 0.3018 - recall: 0.4241 - tn: 5229.6694 - tp: 54.1228
loss :  0.10775098949670792
accuracy :  0.9621949195861816
precision :  0.30000001192092896
recall :  0.4384615421295166
tp :  114.0
fp :  266.0
tn :  10372.0
fn :  146.0
auc :  0.8855164647102356
prc :  0.34701037406921387
f1_beta :  0.40140845743206555


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Xception_05-20-16
341/341 ━━━━━━━━━━━━━━━━━━━━ 77s 192ms/step - accuracy: 0.9732 - auc: 0.8756 - fn: 101.2251 - fp: 45.8801 - loss: 0.1019 - prc: 0.2657 - precision: 0.3889 - recall: 0.2245 - tn: 5310.3159 - tp: 30.4035
loss :  0.10179727524518967
accuracy :  0.9733896255493164
precision :  0.40740740299224854
recall :  0.2538461685180664
tp :  66.0
fp :  96.0
tn :  10542.0
fn :  194.0
auc :  0.8869107961654663
prc :  0.28320854902267456
f1_beta :  0.27454244261302463


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Dataset 2019 evaluation

In [ ]:
for model, model_name in zip(models, MODEL_NAMES):
  print(model_name, "\n")
  results = ModelEvaluating(test_ds19,
                           model, model_name=model_name,
                           metrics=METRICS)
  test_ds19.unbatch().batch(BATCH_SIZE)

EfficientNetV2B0_04-23-10 

258/258 ━━━━━━━━━━━━━━━━━━━━ 23s 90ms/step - accuracy: 0.8614 - auc: 0.7986 - fn: 372.0193 - fp: 213.2355 - loss: 0.3724 - prc: 0.5564 - precision: 0.6007 - recall: 0.4555 - tn: 3275.1389 - tp: 299.3436
loss :  0.36932313442230225
accuracy :  0.8589463233947754
precision :  0.5829145908355713
recall :  0.4370761215686798
tp :  580.0
fp :  415.0
tn :  6496.0
fn :  747.0
auc :  0.8003969788551331
prc :  0.5448529720306396
f1_beta :  0.4600983769962297


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

EfficientNetV2B0_04-29-19 

258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - accuracy: 0.8580 - auc: 0.8075 - fn: 294.8842 - fp: 307.8147 - loss: 0.4347 - prc: 0.5893 - precision: 0.5609 - recall: 0.5652 - tn: 3181.1274 - tp: 375.9112
loss :  0.43828874826431274
accuracy :  0.8538480401039124
precision :  0.5456570386886597
recall :  0.5538809299468994
tp :  735.0
fp :  612.0
tn :  6299.0
fn :  592.0
auc :  0.8100041747093201
prc :  0.5732771158218384
f1_beta :  0.5522163798223042


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Xception_05-20-16 

258/258 ━━━━━━━━━━━━━━━━━━━━ 50s 193ms/step - accuracy: 0.8336 - auc: 0.6791 - fn: 430.4247 - fp: 269.9305 - loss: 0.5487 - prc: 0.4180 - precision: 0.4888 - recall: 0.3742 - tn: 3218.5444 - tp: 240.8378
loss :  0.5423714518547058
accuracy :  0.8296916484832764
precision :  0.4616161584854126
recall :  0.3443858325481415
tp :  457.0
fp :  533.0
tn :  6378.0
fn :  870.0
auc :  0.6751196980476379
prc :  0.39650171995162964
f1_beta :  0.36281359109000233


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir / "test",
  label_mode=None,
  image_size=IM_SIZE,
  batch_size=BATCH_SIZE,
  shuffle=False)

prediction_df = ModelPredicting(test_dataset, best_ft_model, model_name=f"{label}"+ MODEL_NAME)

### Після обробки

In [ ]:
# обробка тестових даних
prepocess_to_crop(pathlib.Path(folder_20) / "test", center_ratio=0.5, threshold=70)

test_ds20 = tf.keras.utils.image_dataset_from_directory(
  pathlib.Path(folder_20) / "test",
  class_names=["other", "melanoma"],
  seed=123,
  # image_size=(IM_SIZE, IM_SIZE),
  batch_size=BATCH_SIZE)

prepocess_to_crop(pathlib.Path(folder_19) / "test", center_ratio=0.5, threshold=70)

test_ds19 = tf.keras.utils.image_dataset_from_directory(
  pathlib.Path(folder_19) / "test",
  class_names=["other", "melanoma"],
  seed=123,
  # image_size=(IM_SIZE, IM_SIZE),
  batch_size=BATCH_SIZE)

With vignette /content/2020_supplemented_cropped/test/other: 23
Without vignette /content/2020_supplemented_cropped/test/other: 10615
10638
With vignette /content/2020_supplemented_cropped/test/melanoma: 1
Without vignette /content/2020_supplemented_cropped/test/melanoma: 259
260
Found 10898 files belonging to 2 classes.
With vignette /content/2019_all_croppedtr/test/other: 2005
Without vignette /content/2019_all_croppedtr/test/other: 4906
6911
With vignette /content/2019_all_croppedtr/test/melanoma: 516
Without vignette /content/2019_all_croppedtr/test/melanoma: 811
1327
Found 8238 files belonging to 2 classes.


Датасет 2020

In [ ]:
for model, model_name in zip(models, MODEL_NAMES):
  print(model_name)
  results = ModelEvaluating(test_ds20,
                           model, model_name=model_name,
                           metrics=METRICS)
  test_ds20.unbatch().batch(BATCH_SIZE)

EfficientNetV2B0_04-23-10
341/341 ━━━━━━━━━━━━━━━━━━━━ 16s 46ms/step - accuracy: 0.9773 - auc: 0.8661 - fn: 103.9795 - fp: 24.2632 - loss: 0.0876 - prc: 0.3043 - precision: 0.5550 - recall: 0.2121 - tn: 5331.5029 - tp: 28.0789
loss :  0.08618088811635971
accuracy :  0.9764177203178406
precision :  0.5128205418586731
recall :  0.23076923191547394
tp :  60.0
fp :  57.0
tn :  10581.0
fn :  200.0
auc :  0.8854691386222839
prc :  0.2965346872806549
f1_beta :  0.2592912731696192


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

EfficientNetV2B0_04-29-19
341/341 ━━━━━━━━━━━━━━━━━━━━ 16s 46ms/step - accuracy: 0.9625 - auc: 0.8821 - fn: 78.0877 - fp: 125.9444 - loss: 0.1091 - prc: 0.3530 - precision: 0.3018 - recall: 0.4241 - tn: 5229.6694 - tp: 54.1228
loss :  0.10786662995815277
accuracy :  0.9621949195861816
precision :  0.30000001192092896
recall :  0.4384615421295166
tp :  114.0
fp :  266.0
tn :  10372.0
fn :  146.0
auc :  0.8853908777236938
prc :  0.34545767307281494
f1_beta :  0.40140845743206555


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Xception_05-20-16
341/341 ━━━━━━━━━━━━━━━━━━━━ 59s 173ms/step - accuracy: 0.9730 - auc: 0.8754 - fn: 101.2251 - fp: 47.5585 - loss: 0.1021 - prc: 0.2634 - precision: 0.3819 - recall: 0.2245 - tn: 5308.6372 - tp: 30.4035
loss :  0.1020938977599144
accuracy :  0.9731143116950989
precision :  0.4000000059604645
recall :  0.2538461685180664
tp :  66.0
fp :  99.0
tn :  10539.0
fn :  194.0
auc :  0.886648416519165
prc :  0.28117093443870544
f1_beta :  0.27385893538182465


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Датасет 2019

In [ ]:
for model, model_name in zip(models, MODEL_NAMES):
  print(model_name)
  results = ModelEvaluating(test_ds19,
                           model, model_name=model_name,
                           metrics=METRICS)
  test_ds19.unbatch().batch(BATCH_SIZE)

EfficientNetV2B0_04-23-10
258/258 ━━━━━━━━━━━━━━━━━━━━ 16s 61ms/step - accuracy: 0.8607 - auc: 0.8457 - fn: 317.4016 - fp: 272.1313 - loss: 0.3471 - prc: 0.5858 - precision: 0.5792 - recall: 0.5378 - tn: 3216.2432 - tp: 353.9614
loss :  0.34389546513557434
accuracy :  0.8591891527175903
precision :  0.568163275718689
recall :  0.5244913101196289
tp :  696.0
fp :  529.0
tn :  6382.0
fn :  631.0
auc :  0.8454910516738892
prc :  0.5804567337036133
f1_beta :  0.5326802210482676


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

EfficientNetV2B0_04-29-19
258/258 ━━━━━━━━━━━━━━━━━━━━ 16s 61ms/step - accuracy: 0.8545 - auc: 0.8633 - fn: 236.1815 - fp: 380.6989 - loss: 0.3771 - prc: 0.6183 - precision: 0.5414 - recall: 0.6497 - tn: 3108.2432 - tp: 434.6139
loss :  0.3782336711883545
accuracy :  0.851177453994751
precision :  0.5313859581947327
recall :  0.644310474395752
tp :  855.0
fp :  754.0
tn :  6157.0
fn :  472.0
auc :  0.862928569316864
prc :  0.6126772165298462
f1_beta :  0.6180425048437344


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Xception_05-20-16
258/258 ━━━━━━━━━━━━━━━━━━━━ 45s 173ms/step - accuracy: 0.8272 - auc: 0.7562 - fn: 358.3822 - fp: 376.0811 - loss: 0.4596 - prc: 0.4234 - precision: 0.4733 - recall: 0.4771 - tn: 3112.3938 - tp: 312.8803
loss :  0.46198031306266785
accuracy :  0.8211944699287415
precision :  0.44640234112739563
recall :  0.4581763446331024
tp :  608.0
fp :  754.0
tn :  6157.0
fn :  719.0
auc :  0.74654620885849
prc :  0.4075984060764313
f1_beta :  0.4557721177651083


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### ViT

In [ ]:
# Transformers package uses Keras 2 objects, current version is Keras 3, packed in Tensorflow since version 2.16.
# Should you want tf.keras to stay on Keras 2 after upgrading to TensorFlow 2.16+, you can configure your TensorFlow installation so that tf.keras points to tf_keras. To achieve this:
# Make sure to install tf_keras. Note that TensorFlow does not install it by default.
# Export the environment variable TF_USE_LEGACY_KERAS=1.

import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
# https://drive.google.com/file/d/1w9vCnxTaL8rbY5UJFWSpFKfLFpfEhppo/view?usp=sharing
file_id = "1w9vCnxTaL8rbY5UJFWSpFKfLFpfEhppo"
url = f"https://drive.google.com/uc?id={file_id}"
output = "ft-VIT-05-04-07.keras"

gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1w9vCnxTaL8rbY5UJFWSpFKfLFpfEhppo
From (redirected): https://drive.google.com/uc?id=1w9vCnxTaL8rbY5UJFWSpFKfLFpfEhppo&confirm=t&uuid=c061e3d0-eb06-4714-9fa9-90d0fc00a44e
To: /content/ft-VIT-05-04-07.keras
100%|██████████| 1.04G/1.04G [00:20<00:00, 49.6MB/s]


'ft-VIT-05-04-07.keras'

In [ ]:
!pip install transformers

In [ ]:
from transformers import TFViTModel, ViTConfig  #,AutoImageProcessor

In [ ]:
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable()
class ExtractCLSToken(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        return inputs[:, 0, :]

    def get_config(self):
        base_config = super().get_config()
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


@register_keras_serializable()
class NormalizeMinus1To1(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, img):
        return (tf.cast(img, tf.float32) / 255.0 - 0.5) * 2.0

    def get_config(self):
        return super().get_config()

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
model_id = "DunnBC22/vit-base-patch16-224-in21k_brain_tumor_diagnosis"

# Нормалізація [-1, 1], зміна розміру
resize_rescale_for_HF_VIT = tf.keras.Sequential([
  layers.Resizing(224, 224),
  NormalizeMinus1To1(),
  layers.Permute((3,1,2)),
], name="resize_rescale_for_HF_VIT")

In [ ]:
custom_objects = {
    "ExtractCLSToken": ExtractCLSToken,
    "resize_rescale_for_HF_VIT": resize_rescale_for_HF_VIT,
    "NormalizeMinus1To1": NormalizeMinus1To1,
    "TFViTModel": TFViTModel
}

In [ ]:
MODEL_NAME = 'ft-VIT-05-04-07'

model_path = '/content/ft-VIT-05-04-07.keras'
ft_model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

TypeError: <class 'tf_keras.src.engine.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'tf_keras.src.engine.functional', 'class_name': 'Functional', 'config': {'name': 'model', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 256, 256, 3], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_1'}, 'registered_name': None, 'name': 'input_1', 'inbound_nodes': []}, {'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'resize_rescale_for_HF_VIT', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, 256, 256, 3], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'resizing_input'}, 'registered_name': None}, {'module': '

In [ ]:
ModelEvaluating(test_ds20,
                ft_model, model_name='VIT-05-04-07',
                metrics=METRICS)


In [ ]:
ModelEvaluating(test_ds19,
                ft_model, model_name='VIT-05-04-07',
                metrics=METRICS)

### На оброблених даних

In [ ]:
ModelEvaluating(test_ds20,
                ft_model, model_name='VIT-05-04-07',
                metrics=METRICS)


In [ ]:
ModelEvaluating(test_ds19,
                ft_model, model_name='VIT-05-04-07',
                metrics=METRICS)

### Оцінка моделі, що навчалась на наборі №1 на змішаних тестових даних

In [ ]:
model_path = '/content/ft-EfficientNetV2B0_04-12-10_last.keras'

# efficientnet_v2_preprocess, densenet_preprocess, xception_preprocess
model = keras.models.load_model(model_path, custom_objects={"preprocess_input": efficientnet_v2_preprocess})

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir / "test",
  class_names=["other", "melanoma"],
  seed=123,
  # image_size=(IM_SIZE, IM_SIZE),
  batch_size=32)

Found 19136 files belonging to 2 classes.


In [ ]:
results = ModelEvaluating(test_ds,
                          model, model_name="EfficientNetV2B0_04-12-10_last",
                          metrics=METRICS)

598/598 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.6888 - auc: 0.8064 - fn: 111.3973 - fp: 2863.2505 - loss: 3.1328 - prc: 0.2005 - precision: 0.1920 - recall: 0.8603 - tn: 5951.1021 - tp: 674.1970
loss :  3.123044967651367
accuracy :  0.6893812417984009
precision :  0.1911243498325348
recall :  0.8494014143943787
tp :  1348.0
fp :  5705.0
tn :  11844.0
fn :  239.0
auc :  0.8037760257720947
prc :  0.20061124861240387
f1_beta :  0.5029475568493469


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# обробка тестових даних
prepocess_to_crop(data_dir / "test", center_ratio=0.5, threshold=70)

test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir / "test",
  class_names=["other", "melanoma"],
  seed=123,
  # image_size=(IM_SIZE, IM_SIZE),
  batch_size=32)

With vignette /content/full_ds_croppedtr_no_augm/test/other: 2028
Without vignette /content/full_ds_croppedtr_no_augm/test/other: 15521
17549
With vignette /content/full_ds_croppedtr_no_augm/test/melanoma: 517
Without vignette /content/full_ds_croppedtr_no_augm/test/melanoma: 1070
1587
Found 19136 files belonging to 2 classes.


оцінка після обробки

In [ ]:
results = ModelEvaluating(test_ds,
                          model, model_name="EfficientNetV2B0_04-12-10_last",
                          metrics=METRICS)

598/598 ━━━━━━━━━━━━━━━━━━━━ 27s 45ms/step - accuracy: 0.6880 - auc: 0.8056 - fn: 113.4541 - fp: 2868.9666 - loss: 3.2503 - prc: 0.1996 - precision: 0.1913 - recall: 0.8581 - tn: 5945.3857 - tp: 672.1403
loss :  3.2422122955322266
accuracy :  0.6887019276618958
precision :  0.19042222201824188
recall :  0.8468809127807617
tp :  1344.0
fp :  5714.0
tn :  11835.0
fn :  243.0
auc :  0.8024864196777344
prc :  0.19925439357757568
f1_beta :  0.5012680988756599


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>